# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import string
import numpy as np
import pandas as pd
import pickle
from sqlalchemy import create_engine
import nltk
nltk.download(['wordnet', 'punkt', 'stopwords'])
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import  SVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponseETL.db')
df = pd.read_sql_table('DisasterResponseETL', engine)
X = df['message']
y = df[df.columns[4:]]

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    """
    Input: messages
    
    Output: list of words after processing the following steps
    """
    
    # The following lines of code replace url link with "urlplaceholder"
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    # The following lines of code lemmatizes words
    """
    Examples of lemmatization:
    -> rocks : rock
    -> corpora : corpus
    -> better : good
    """
    tokens = word_tokenize(text)
    
    # Remove stopwords
    tokens = [w for w in tokens if w not in stopwords.words("english")]
    
    # Lemmatize words
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip() # Convert to lower case and remove whitespace
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
                   ('vect', CountVectorizer(tokenizer=tokenize)),
                   ('tfidf', TfidfTransformer()),
                   ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
np.random.seed(0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [6]:
# Pickle best model
pickle.dump(pipeline, open('DisasterResponseModel.pkl', 'wb'))

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
# Test model using the test set
y_pred = pipeline.predict(X_test)

In [51]:
for i in range(y.shape[1]):
    print(y.columns[i], ':')
    print(classification_report(y_test.values[:,i], y_pred[:,i]), '_'*50)

('related',) :
             precision    recall  f1-score   support

          0       0.64      0.34      0.45      1266
          1       0.81      0.94      0.87      3938
          2       0.67      0.05      0.09        40

avg / total       0.77      0.79      0.76      5244
 __________________________________________________
('request',) :
             precision    recall  f1-score   support

          0       0.88      0.98      0.93      4349
          1       0.83      0.37      0.51       895

avg / total       0.88      0.88      0.86      5244
 __________________________________________________
('offer',) :
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5218
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      5244
 __________________________________________________
('aid_related',) :
             precision    recall  f1-score   support

          0       0.73      0.88 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [64]:
parameters = {
        'tfidf__use_idf':[True, False],
        'clf__estimator__n_estimators': [10, 25],
        'clf__estimator__min_samples_split': [3, 4]
    }

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=2)
cv.fit(X_train, y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=10, tfidf__use_idf=True 
[CV]  clf__estimator__min_samples_split=3, clf__estimator__n_estimators=10, tfidf__use_idf=True, total=  42.4s
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=10, tfidf__use_idf=True 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   52.3s remaining:    0.0s


[CV]  clf__estimator__min_samples_split=3, clf__estimator__n_estimators=10, tfidf__use_idf=True, total=  42.9s
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=10, tfidf__use_idf=True 
[CV]  clf__estimator__min_samples_split=3, clf__estimator__n_estimators=10, tfidf__use_idf=True, total=  43.0s
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=10, tfidf__use_idf=False 
[CV]  clf__estimator__min_samples_split=3, clf__estimator__n_estimators=10, tfidf__use_idf=False, total=  41.5s
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=10, tfidf__use_idf=False 
[CV]  clf__estimator__min_samples_split=3, clf__estimator__n_estimators=10, tfidf__use_idf=False, total=  42.0s
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=10, tfidf__use_idf=False 
[CV]  clf__estimator__min_samples_split=3, clf__estimator__n_estimators=10, tfidf__use_idf=False, total=  41.7s
[CV] clf__estimator__min_samples_split=3, clf__estimato

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed: 30.7min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'tfidf__use_idf': [True, False], 'clf__estimator__n_estimators': [10, 25], 'clf__estimator__min_samples_split': [3, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [65]:
cv.cv_results_

/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py

{'mean_fit_time': array([ 37.78920531,  36.80704292,  83.52234697,  81.50737007,
         35.42082071,  35.25600568,  79.28491489,  77.24178457]),
 'std_fit_time': array([ 0.27793729,  0.16187177,  0.25739382,  0.38371875,  0.23298933,
         0.20123753,  0.26360202,  0.49882296]),
 'mean_score_time': array([ 4.98119171,  4.90137704,  7.32842986,  7.2088182 ,  4.93664479,
         4.9346323 ,  7.31007028,  7.24318322]),
 'std_score_time': array([ 0.03740526,  0.04557341,  0.0163742 ,  0.05062   ,  0.01821878,
         0.01589166,  0.02320881,  0.03580556]),
 'param_clf__estimator__min_samples_split': masked_array(data = [3 3 3 3 4 4 4 4],
              mask = [False False False False False False False False],
        fill_value = ?),
 'param_clf__estimator__n_estimators': masked_array(data = [10 10 25 25 10 10 25 25],
              mask = [False False False False False False False False],
        fill_value = ?),
 'param_tfidf__use_idf': masked_array(data = [True False True False Tru

In [66]:
cv.best_params_

{'clf__estimator__min_samples_split': 3,
 'clf__estimator__n_estimators': 25,
 'tfidf__use_idf': False}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [67]:
y_pred_improved = cv.predict(X_test)
for i in range(y.shape[1]):
    print(y.columns[i], ':')
    print(classification_report(y_test.values[:,i], y_pred_improved[:,i]), '_'*50)

('related',) :
             precision    recall  f1-score   support

          0       0.71      0.25      0.37      1266
          1       0.79      0.97      0.87      3938
          2       0.60      0.07      0.13        40

avg / total       0.77      0.79      0.75      5244
 __________________________________________________
('request',) :
             precision    recall  f1-score   support

          0       0.89      0.99      0.93      4349
          1       0.85      0.40      0.54       895

avg / total       0.88      0.88      0.87      5244
 __________________________________________________
('offer',) :
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5218
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      5244
 __________________________________________________
('aid_related',) :
             precision    recall  f1-score   support

          0       0.76      0.87 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [68]:
# using SVM instead 
pipeline_SVM = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(SVC()))
])

parameters_SVM = {'vect__min_df': [5],
              'tfidf__use_idf':[True],
              'clf__estimator__kernel': ['poly'], 
              'clf__estimator__degree': [2, 3],
              'clf__estimator__C':[10, 100]}

cv2 = GridSearchCV(pipeline_SVM, param_grid=parameters_SVM, verbose=2)

# Find best parameters
np.random.seed(77)
SVM_model = cv2.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__C=10, clf__estimator__degree=2, clf__estimator__kernel=poly, tfidf__use_idf=True, vect__min_df=5 


ValueError: The number of classes has to be greater than one; got 1

### 9. Export your model as a pickle file

In [69]:
# Pickle best model
pickle.dump(cv, open('disaster_response_model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.